In [26]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

In [16]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand what love is.")
]

In [17]:
res = chat(messages)
print(res.content)

/Users/zaidsaheb/Desktop/projects/xppcoder/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Love is a complex and multifaceted emotion that can be experienced in many different ways. It involves feelings of deep affection, care, and attachment towards another person. Love can manifest in various forms, such as romantic love, platonic love, familial love, and self-love. It often involves a strong emotional connection, empathy, and a desire to support and nurture the well-being of the person you love. Love can bring joy, fulfillment, and a sense of belonging, but it can also be challenging and require effort and compromise to maintain. Ultimately, love is a deeply personal and subjective experience that can have a profound impact on our lives.


In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Do you believe in love?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

In [22]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What does page 69 of Extending Microsoft Dynamics 365 Finance and Supply Chain Management Cookbook Second Edition by Simon Buxton talk about?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [23]:
print(res.content)

I don't have access to specific pages or content from the book "Extending Microsoft Dynamics 365 Finance and Supply Chain Management Cookbook Second Edition" by Simon Buxton. However, I can provide a general overview of the topics covered in the book.

The book is a practical guide that focuses on extending and customizing Microsoft Dynamics 365 Finance and Supply Chain Management using various tools and techniques. It covers a wide range of topics related to development, customization, and integration with the Dynamics 365 platform.

Some of the key topics covered in the book may include:

- Customizing forms, tables, and reports in Dynamics 365 Finance and Supply Chain Management
- Extending functionality using X++, C#, and .NET technologies
- Integrating with external systems and services
- Implementing business logic and workflows
- Enhancing security and data management
- Performance optimization and best practices for development

If you are looking for specific information from 

In [27]:
from pinecone import Pinecone

import re
import pdfplumber
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("xppcoder")

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')
MODEL = "text-embedding-ada-002"

In [ ]:
# Define a function to preprocess text
def preprocess_text(text):
    # Replace consecutive spaces, newlines and tabs
    text = re.sub(r'\s+', ' ', text)
    return text

def process_pdf(file_path):
    # create a loader
    loader = PyPDFLoader(file_path)
    # load your data
    data = loader.load()
    # Split your data up into smaller documents with Chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    documents = text_splitter.split_documents(data)
    # Convert Document objects into strings
    texts = [str(doc) for doc in documents]
    return texts

# Define a function to create embeddings
def create_embeddings(texts):
    embeddings_list = []
    for text in texts:
        res = openai.Embedding.create(input=[text], engine=MODEL)
        embeddings_list.append(res['data'][0]['embedding'])
    return embeddings_list

# Define a function to upsert embeddings to Pinecone
def upsert_embeddings_to_pinecone(index, embeddings, ids):
    index.upsert(vectors=[(id, embedding) for id, embedding in zip(ids, embeddings)])

# Process a PDF and create embeddings
file_path = "fnobook.pdf"  # Replace with your actual file path
texts = process_pdf(file_path)
embeddings = create_embeddings(texts)

# Upsert the embeddings to Pinecone
upsert_embeddings_to_pinecone(index, embeddings, [file_path])